# Graph Convolutional Network by Kipf and Welling

## Imports

In [16]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle
import numpy as np

import itertools

## GNN Definition

In [17]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [18]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(1433, 21, F.relu)
        self.gcn2 = GCN(21, 7, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        #x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

## Data Loading

In [20]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph

# add self loop
g.remove_edges_from(nx.selfloop_edges(g))
g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

## Select Training Set

In [21]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [22]:
label_perms = list(itertools.permutations(np.unique(labels)))

In [23]:
import time

net = Net()
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(150):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(g, features)
    #scores = th.tensor([F.nll_loss(prediction.detach()[1-mask][:,p], labels[1-mask]) for p in label_perms])
    #val_loss = th.min(scores)
    #val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    val_loss=th.tensor(np.nan)
    net.train()

    # Compute loss for train nodes
    logits = net(g, features)

    loss = th.tensor(1000.0,requires_grad=True)
    for p in label_perms:
        loss = th.min(loss,F.nll_loss(logits[mask][:,p], labels[mask]))

    #loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Val.Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), val_loss.item(), np.mean(dur)))

Epoch 00000 | Loss 1.8666 | Val.Loss nan | Time(s) nan
Epoch 00001 | Loss 1.8236 | Val.Loss nan | Time(s) nan
Epoch 00002 | Loss 1.7831 | Val.Loss nan | Time(s) nan
Epoch 00003 | Loss 1.7413 | Val.Loss nan | Time(s) 1.0281
Epoch 00004 | Loss 1.7013 | Val.Loss nan | Time(s) 1.0271
Epoch 00005 | Loss 1.6637 | Val.Loss nan | Time(s) 1.0298
Epoch 00006 | Loss 1.6283 | Val.Loss nan | Time(s) 1.0316
Epoch 00007 | Loss 1.5954 | Val.Loss nan | Time(s) 1.0323
Epoch 00008 | Loss 1.5657 | Val.Loss nan | Time(s) 1.0676
Epoch 00009 | Loss 1.5394 | Val.Loss nan | Time(s) 1.0754
Epoch 00010 | Loss 1.5147 | Val.Loss nan | Time(s) 1.0860
Epoch 00011 | Loss 1.4918 | Val.Loss nan | Time(s) 1.0931
Epoch 00012 | Loss 1.4702 | Val.Loss nan | Time(s) 1.0873
Epoch 00013 | Loss 1.4501 | Val.Loss nan | Time(s) 1.0879
Epoch 00014 | Loss 1.4310 | Val.Loss nan | Time(s) 1.0835
Epoch 00015 | Loss 1.4129 | Val.Loss nan | Time(s) 1.0784
Epoch 00016 | Loss 1.3954 | Val.Loss nan | Time(s) 1.0760
Epoch 00017 | Loss 1.37

In [26]:
# Visualise predictions
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
a = np.transpose(np.vstack([final_prediction[mask].numpy().argmax(axis=1),labels[mask].numpy()]))
a[a[:,0].argsort()]
# as can be seen, the net predicts other labels, but gets the clusters right :)

array([[0, 4],
       [0, 4],
       [0, 4],
       [0, 4],
       [0, 4],
       [0, 4],
       [0, 4],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [2, 5],
       [2, 5],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [3, 3],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [4, 2],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [5, 1],
       [6, 6],
       [6, 6],
       [6, 6],
       [6, 6],
       [6, 6],
       [6, 6],
       [6, 6]], dtype=int64)

## Evaluation

In [27]:
from sklearn.metrics import accuracy_score as acc
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()

pred_sets = {"All ":final_prediction,"Train":final_prediction[mask],"Test":final_prediction[1-mask]}
label_sets = {"All ":labels,"Train":labels[mask],"Test":labels[1-mask]}
eval_functions = {"NLL-Loss":lambda y,x: F.nll_loss(x,y),"Accuracy":lambda y,x: acc(y,x.numpy().argmax(axis=1))}

for name,func in eval_functions.items():
    eval_message = f"\n{name}:\n"
    for subset in pred_sets.keys():
        eval_message += f" {subset}: {func(label_sets[subset],pred_sets[subset]):.4f} |"
    print(eval_message)


NLL-Loss:
 All : 3.9184 | Train: 4.2528 | Test: 3.9116 |

Accuracy:
 All : 0.2097 | Train: 0.3333 | Test: 0.2072 |
